In [6]:
#!pip install langchain
#!pip install langchain-community
#!pip install faiss-cpu
#!pip install openai
#!pip install sentence-transformers
#!pip install tf-keras

In [ ]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

vector_store_path = "C:/Users/Steffen/Desktop/vector_store_1"

# 1. Load .txt files from 'documents' folder
def load_documents(folder_path="C:/Users/Steffen/Dropbox"):
    docs = []
    for file in os.listdir(folder_path):
        if file.startswith("lex-home") and file.endswith(".txt"):
            path = os.path.join(folder_path, file)
            loader = TextLoader(path, encoding='utf-8')
            docs.extend(loader.load())
    return docs

# 2. Split documents into chunks
def split_documents(documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size=240, chunk_overlap=30)
    return splitter.split_documents(documents)


# 4. Create a RAG chain using OpenAI GPT
def create_qa_chain(vectorstore):
    retriever = vectorstore.as_retriever()
    llm = ChatOpenAI(temperature=0.0, model_name="gpt-3.5-turbo")  # OpenAI API key must be set
    return RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# 5. Main loop
def main():
    print("Loading documents...")
    docs = load_documents()
    print(f"Loaded {len(docs)} documents.")

    print("Splitting into chunks...")
    chunks = split_documents(docs)
    print(f"{len(chunks)} chunks created.")
    
    print("Creating vector index...")
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    use_local = input("Use local embeddings? (yes/no): ").lower().strip()

    if use_local == "yes" and os.path.exists(vector_store_path):
        print("Loading existing vector store...")
        vectorstore = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
        print("Vector store loaded successfully!")
    else:
        print("Creating new vector store...")
        vectorstore = FAISS.from_documents(chunks, embeddings)
        vectorstore.save_local(vector_store_path)
    
    print("Initializing question-answering chain...")
    with open("C:/Users/Steffen/Desktop/openai_key.txt", "r") as f:
        key = f.read().strip()
    os.environ["OPENAI_API_KEY"] = key
    qa = create_qa_chain(vectorstore)

    print("\nReady! Ask questions about your documents.")
    while True:
        query = input("\nYour question ('exit' to quit): ")
        if query.lower() in ["exit", "quit"]:
            break
        answer = qa.run(query)
        print("\nAnswer:", answer)
    
if __name__ == "__main__":
    main()

Loading documents...
Loaded 1 documents.
Splitting into chunks...
11749 chunks created.
Creating vector index...


C:\Users\Steffen\AppData\Local\Temp\ipykernel_11760\4053157642.py:42: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")



Initializing question-answering chain...


C:\Users\Steffen\AppData\Local\Temp\ipykernel_11760\4053157642.py:28: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.0, model_name="gpt-3.5-turbo")  # OpenAI API key must be set



Ready! Ask questions about your documents.
